## Analysis Data Export
Load the evaluation results and the participant information. Add the LLM as a participant (optional) and merge all data into a single dataframe.

In [ ]:
from service.json_service import load_evaluation_progress, load_common_evaluation_config
import pandas as pd
import os


evaluation_config_path = "data/4_expert_evaluation/output_depseudonymized/common_evaluation_config.json"
evaluation_progress_path = "data/4_expert_evaluation/output_depseudonymized/"

# Set this if you want to include the llm's evaluation.
# Per default, the 5_llm_as_a_judge notebook stores the evaluation progress in "data/5_llm_evaluation/evaluation_progress_llm-as-a-judge.json"
llm_evaluation_progress_path = None 

participant_info_path = "data/6_analysis/participant_info.csv"

analysis_output_path = "data/6_analysis/"

# Load evaluation progress
df = load_evaluation_progress(evaluation_progress_path, llm_evaluation_progress_path)

# Load evaluation config
evaluation_config_df = load_common_evaluation_config(evaluation_config_path)
df = df.merge(evaluation_config_df, on=["exercise_id", "submission_id", "feedback_type"], how="left")

# Load participant info and add LLM as a participant
participant_info_df = pd.read_csv(participant_info_path, delimiter=";")
if llm_evaluation_progress_path:
    participant_info_df = pd.concat([participant_info_df, pd.DataFrame([{
        'expert_id': 'llm',
        'evaluation_name': 'LLM as a judge',
        'link': '',
        'name': 'LLM',
        'study_program': '',
        'semester': pd.NA,
        'eist_participation': pd.NA,
        'pse_participation': pd.NA,
        'tutoring_experience': pd.NA,
        'group': 'LLM',
    }])])
df = df.merge(participant_info_df, on=["expert_id"], how="left")

### Export Data
Select the relevant columns and export the data to a CSV file.

In [ ]:
df = df[['expert_id', 'study_program', 'semester', 'eist_participation', 'pse_participation', 'tutoring_experience', 'group', 'exercise_id', 'submission_id', 'feedback_type', 'metric', 'score', 'exercise', 'submission', 'feedback']]

os.makedirs(analysis_output_path, exist_ok=True)
df.to_csv(os.path.join(analysis_output_path, "evaluation_data.csv"), index=False)